import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [57]:
from keras.models import Model
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Lambda
from keras.applications import inception_resnet_v2
# from keras.preprocessing.image import *

import numpy as np
from tqdm import tqdm
import cv2

In [58]:
n = 25000
img_height, img_width = 299, 299
X = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

for i in tqdm(range(12500)): # n/2
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' % i), (img_height, img_width))
    X[i+12500] = cv2.resize(cv2.imread('train/dog.%d.jpg' % i), (img_height, img_width)) # n/2

# 打上标签
y[12500:] = 1

100%|██████████| 12500/12500 [00:57<00:00, 218.61it/s]


In [59]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [60]:
i_input_tensor = Input((img_height, img_width, 3))
# x_input_tensor = inception_resnet_v2.preprocess_input(i_input_tensor)	# 错误的处理
x_input_tensor = Lambda(inception_resnet_v2.preprocess_input)(i_input_tensor) # 建立模型预处理

# 搭建基础模型
base_model = inception_resnet_v2.InceptionResNetV2(input_tensor=x_input_tensor, weights='imagenet', include_top=False)

范例：不运行
from keras.callbacks import ModelCheckpoint

'''
如果验证损失下降， 那么在每个训练轮之后保存模型。
'''
checkpointer = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])

## RUN BELOW

### 下面修改

In [61]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='weights_checkpint_060_770.hdf5', verbose=1, save_best_only=True)

### 下面修改

In [62]:
# 锁定所有神经网络层
for layers in base_model.layers:
    layers.trainable = False

i_output = GlobalAveragePooling2D()(base_model.output)
i_output = Dropout(0.6)(i_output) # 0.25 比 0.5 好
i_predictions = Dense(1, activation='sigmoid')(i_output)
model = Model(base_model.input, i_predictions)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [63]:
model.fit(X_train, y_train, batch_size=20,     # defalt 16
	epochs=8, 	# defalt 5
	validation_data=(X_valid, y_valid),
         verbose=1, callbacks=[checkpointer])


Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 168s 8ms/step - loss: 0.1696 - acc: 0.9370 - val_loss: 0.0768 - val_acc: 0.9776

Epoch 00001: val_loss improved from inf to 0.07681, saving model to weights_checkpint_060_770.hdf5
Epoch 2/8
20000/20000 [==============================] - 148s 7ms/step - loss: 0.1141 - acc: 0.9549 - val_loss: 0.0694 - val_acc: 0.9818

Epoch 00002: val_loss improved from 0.07681 to 0.06941, saving model to weights_checkpint_060_770.hdf5
Epoch 3/8
20000/20000 [==============================] - 148s 7ms/step - loss: 0.1101 - acc: 0.9593 - val_loss: 0.0884 - val_acc: 0.9750

Epoch 00003: val_loss did not improve from 0.06941
Epoch 4/8
20000/20000 [==============================] - 148s 7ms/step - loss: 0.1076 - acc: 0.9601 - val_loss: 0.0966 - val_acc: 0.9734

Epoch 00004: val_loss did not improve from 0.06941
Epoch 5/8
20000/20000 [==============================] - 148s 7ms/step - loss: 0.1020 - acc: 0.

In [64]:
for i in range(len(model.layers)):
    print(model.layers[i].name, i)

input_5 0
lambda_5 1
conv2d_813 2
batch_normalization_813 3
activation_813 4
conv2d_814 5
batch_normalization_814 6
activation_814 7
conv2d_815 8
batch_normalization_815 9
activation_815 10
max_pooling2d_17 11
conv2d_816 12
batch_normalization_816 13
activation_816 14
conv2d_817 15
batch_normalization_817 16
activation_817 17
max_pooling2d_18 18
conv2d_821 19
batch_normalization_821 20
activation_821 21
conv2d_819 22
conv2d_822 23
batch_normalization_819 24
batch_normalization_822 25
activation_819 26
activation_822 27
average_pooling2d_5 28
conv2d_818 29
conv2d_820 30
conv2d_823 31
conv2d_824 32
batch_normalization_818 33
batch_normalization_820 34
batch_normalization_823 35
batch_normalization_824 36
activation_818 37
activation_820 38
activation_823 39
activation_824 40
mixed_5b 41
conv2d_828 42
batch_normalization_828 43
activation_828 44
conv2d_826 45
conv2d_829 46
batch_normalization_826 47
batch_normalization_829 48
activation_826 49
activation_829 50
conv2d_825 51
conv2d_827 52

### 下面

In [65]:
# 放开靠后部分网络层进行 fine tune 微调模型
for layer in model.layers[770:]:	# total 783 layers
    layer.trainable = True


from keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [66]:
model.fit(X_train, y_train, batch_size=20, 	# first: 16
	epochs=8, 	# first: 5 
	validation_data=(X_valid, y_valid),
         verbose=1, callbacks=[checkpointer])

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 172s 9ms/step - loss: 0.0956 - acc: 0.9648 - val_loss: 0.0413 - val_acc: 0.9908

Epoch 00001: val_loss improved from 0.05668 to 0.04128, saving model to weights_checkpint_060_770.hdf5
Epoch 2/8
20000/20000 [==============================] - 150s 8ms/step - loss: 0.0940 - acc: 0.9646 - val_loss: 0.0413 - val_acc: 0.9908

Epoch 00002: val_loss did not improve from 0.04128
Epoch 3/8
20000/20000 [==============================] - 151s 8ms/step - loss: 0.0895 - acc: 0.9682 - val_loss: 0.0422 - val_acc: 0.9908

Epoch 00003: val_loss did not improve from 0.04128
Epoch 4/8
20000/20000 [==============================] - 150s 8ms/step - loss: 0.0869 - acc: 0.9677 - val_loss: 0.0402 - val_acc: 0.9908

Epoch 00004: val_loss improved from 0.04128 to 0.04017, saving model to weights_checkpint_060_770.hdf5
Epoch 5/8
20000/20000 [==============================] - 151s 8ms/step - loss: 0.0886 - acc

### 下面修改

In [67]:
model.save('modelInceptionResNet_transfer_Doup060_770layer.h5')

读取测试图片数据

In [68]:
X_test = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
for i in tqdm(range(12500)):
    j = i+1
    X_test[i] = cv2.resize(cv2.imread('test/%d.jpg' % j), (img_height, img_width))

100%|██████████| 12500/12500 [00:28<00:00, 438.52it/s]


In [69]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

25000/25000 [==============================] - 154s 6ms/step


### 下面修改

In [70]:
import pandas as pd
df = pd.read_csv("sample_submission.csv")

for i in range(12500):
	df.set_value(i, 'label', y_pred[i])

df.to_csv('pred_changePredictCsvMethond_Drop060_770layer.csv', index=None)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
